In [2]:
# pip install requests
# pip install -U beautifulsoup4
# pip install pandas
# pip install selenium
# pip install webdriver_manager
# pip install openpyxl

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4085358481.py, line 1)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import warnings
import re
warnings.filterwarnings("ignore")
# Инициализация опций Chrome
chrome_options = Options() # импорт базовых опций chrome
chrome_options.add_argument("--headless") # отключаем визуальное отображение браузера
# chrome_options.add_argument("--start-maximized") # открытие браузера Chrome в развернутом режиме
chrome_options.add_argument("--disable-gpu") # отключаем работу gpu, на всякий случай, чтобы точно ничего визуально не прогружалось
chrome_options.add_argument("--disable-extensions") # отключение расширений браузера, которые могут нарушать работу кода

Код для парсинга

In [2]:
def to_dt(s):
    m={'января':'01', 'февраля':'02', 'марта':'03', 'апреля':'04', 'мая':'05', 'июня':'06', 'июля':'07', 'августа':'08', 'сентября':'09', 'октября':'10', 'ноября':'11', 'декабря':'12'}
    sl = s.split()
    return datetime.strptime(f'{sl[0]} {m[sl[1]]} {sl[2]}', "%d %m %Y")

categories = ['web-development','seo','context','apps-development','smm','marketing','identity','copywriting','crm','gamedev']
#categories = ['context']
categories_rus = ['Разработка сайтов','Поисковая оптимизация','Реклама','Мобильные приложения','SMM и PR','Маркетинг','Дизайн и брендинг','Контент','CRM, 1C, ПО, чат-боты, администрирование']
price_min = 50000
date_min = datetime.strptime('01.10.2023', "%d.%m.%Y")
statuses = ['published','finished','closed']
df = pd.DataFrame({
    'category':[],
    'status':[],
    'public':[],
    'short_text':[],
    'url':[],
    'cost':[],
    'date_public':[],
    'date_end':[],
    'category':[],
    'text':[],
    'service':[],
    'type':[],
    'requirement_performers':[],
    'participants':[],
    'organizer':[]
})
# подключение драйвера Хром для работы с селениумом
driver = webdriver.Chrome(options = chrome_options)
kart = webdriver.Chrome(options = chrome_options)
# start = time.time()
for category in categories:
    url = f'https://workspace.ru/tenders/{category}/?PRICE_FROM={price_min}&COUNTRY_ID=31&SORT=deadline&ORDER=0'
    driver.get(url)
    try:
        tender_count = int(re.search(r'\d+',driver.find_element(By.CLASS_NAME,'tender-list-header__count').text)[0])
    except:
        tender_count = 0
    print(f'Всего найдено записей {tender_count} в категории {category}')
    pg_exit = False # устовие выхода по дате
    zcount = 0 # количество заказов на странице
    start_pos = 0
    if tender_count > 0:
        for pg in range((tender_count-1)//10):
            if pg_exit:
                break
            try: # ждем загрузки страницы
                element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CLASS_NAME,'tender-list-header__count'))
                )
                titles = driver.find_elements(By.CLASS_NAME,'vacancies__card')
            except:
                print('Page not refreshed')
                break
            if len(titles) > zcount: # количество записей увеличилось
                start_pos = zcount
                zcount = len(titles)
                for cnt  in range(start_pos, zcount): 
                    el = titles[cnt]
                    date_end = to_dt(el.find_element(By.CLASS_NAME,'b-tender__info').text.split('\n')[3])
                    if date_end < date_min:
                        pg_exit = True
                        break
                    pos,cond,participants = {},'',''
                    pos['category'] = category
                    pos['status'] = el.find_element(By.CLASS_NAME,'tendercart-type__status-public').text
                    pos['public'] = el.find_element(By.CLASS_NAME,'reference').text 
                    pos['short_text'] = el.find_element(By.CLASS_NAME,'b-tender__title').text 
                    pos['cost'] = el.find_element(By.CLASS_NAME,'b-tender__info-item-text').text 
                    date_public = to_dt(el.find_element(By.CLASS_NAME,'b-tender__info').text.split('\n')[1])
                    pos['date_public'] = date_public.strftime("%d.%m.%Y")
                    date_end = to_dt(el.find_element(By.CLASS_NAME,'b-tender__info').text.split('\n')[3])
                    pos['date_end'] = date_end.strftime("%d.%m.%Y")
                    pos['category'] = category
                    pos['url'] = el.find_element(By.CLASS_NAME,'b-tender__title').find_element(By.TAG_NAME,'a').get_attribute("href")
                    print(f'Parsing {cnt} element, page {pg}. {pos["date_end"]}  {pos["short_text"]}')
                    if pos['public']!='ПРИВАТНЫЙ':
                    # заходим в карточку
                        kart.get(pos['url'])
                        try: # ждем загрузки страницы
                            element = WebDriverWait(driver, 1).until(
                                EC.presence_of_element_located((By.CLASS_NAME,'tendercard__desc'))
                            )
                        except:
                            pass
                        try: #
                            pos['text'] = kart.find_element(By.CLASS_NAME,'tendercard__desc').text.replace('\n',' ')
                            pos['organizer'] = kart.find_element(By.CLASS_NAME,'card-info__desc').text
                            cond = kart.find_element(By.XPATH,"//div[contains(text(), 'Требуемая услуга')]/parent::div/parent::div").text.split('\n')
                            pos['service'] = cond[cond.index('Требуемая услуга')+1]
                            pos['requirement_performers'] = cond[cond.index('Исполнители')+1]
                            participants = kart.find_element(By.XPATH,"//div[contains(text(), 'В заказе участвуют')]/parent::div")
                            pos['participants'] = {u.text:u.get_attribute("href") for u in participants.find_elements(By.TAG_NAME,'a')}
                            print(f'Parsing {cnt} cart element')
                        except:
                            pass
                    df.loc[len(df)] = pos
                try: # жмем кнопку "Показать ещё 10"
                    el.find_element(By.CLASS_NAME,'box-more__btn').click()
                except: # Если кнопки нет, то последняя страница
                    pg_exit = True
                    break
driver.quit()
kart.quit()

Всего найдено записей 1530 в категории web-development
Parsing 0 element, page 0. 01.03.2024  Разработка web3-эквайринга
Parsing 0 cart element
Parsing 1 element, page 0. 15.02.2024  Закрытый заказ по разработке сайта
Parsing 2 element, page 0. 15.02.2024  Функциональное расписание для отображения времени прибытия и отбытия транспорта
Parsing 2 cart element
Parsing 3 element, page 0. 15.02.2024  Редизайн интернет-магазина косметики
Parsing 3 cart element
Parsing 4 element, page 0. 08.02.2024  Создать инструмент по импорту контента с сайтов поставщиков на сайт интернет-магазина
Parsing 4 cart element
Parsing 5 element, page 0. 08.02.2024  Сделать мобильную версию сайта
Parsing 5 cart element
Parsing 6 element, page 0. 08.02.2024  Разработка интернет-магазина под ключ
Parsing 6 cart element
Parsing 7 element, page 0. 03.02.2024  Исправить некоторые моменты на уже существующем сайте
Parsing 7 cart element
Parsing 8 element, page 0. 01.02.2024  Портал для обучения персонала
Parsing 8 cart 

In [4]:

df.to_excel('workspace.xlsx', index=False)


NameError: name 'df' is not defined